# 1. 설정

In [ ]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install sentence-transformers
!pip install tqdm

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from huggingface_hub import login
import os

In [ ]:
HUGGINGFACE_TOKEN = ""
login(token=HUGGINGFACE_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/your_excel_path.xlsx"
BASE_MODEL = "google/gemma-2b-it"
FINETUNE_MODEL = '/content/drive/MyDrive/Gemma/Model'
ADAPTER_MODEL = "/content/drive/MyDrive/Gemma/lora_adapter"
FEEDBACK_PATH = "/content/drive/MyDrive/Gemma/feedback.csv"

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# 1. CSV 파일을 pandas로 로드합니다.
df = pd.read_excel(DATA_PATH)

# 데이터셋을 훈련 데이터셋과 검증 데이터셋으로 나누기
train_df , test_df = train_test_split(df, test_size=0.1, random_state=42)

# 2. pandas DataFrame을 Hugging Face Dataset으로 변환합니다.
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
test_df.head()

# 2. 모델 테스트

In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

In [ ]:
req = train_dataset[0]['질문']
ans = train_dataset[0]['답변']

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "작성된 자기소개서에 대한 좋은점과 개선점을 논리적으로 피드백 해줘.\n\n 자기소개서 문항:\n\n{}".format(req) + " \n\n자기소개서 답변:\n\n{}".format(ans)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

# 3. 모델 학습

In [ ]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['직무'])):
        # 데이터 샘플 추출
        job = example['직무'][i]
        question = example['질문'][i]
        answer = example['답변'][i]
        bad = example['단점'][i] if example['단점'][i] is not None else ""

        # 프롬프트 문자열 생성
        prompt = (
            f"<bos><start_of_turn>user\n"
            f"자기소개서 문항에 대해서 지원자가 작성한 자기소개서 답변을 지원 직무를 고려하여, 채용 담당자 관점에서 개선점을 피드백 해주세요.\n"
            f"지원 직무: {job}\n"
            f"자기소개서 문항: {question}\n"
            f"자기소개서 답변: {answer}<end_of_turn>\n"
            f"<start_of_turn>model\n"
            f"개선점: {bad}<end_of_turn><eos>"
        )
        prompt_list.append(prompt)

    return prompt_list

In [ ]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
#        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained(FINETUNE_MODEL)

#4. 학습된 모델 테스트

In [ ]:
finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('stsb-roberta-large')  # STS 모델 로드

def evaluate_sts(feedbacks, references):
    # 각 문장의 의미적 임베딩 벡터 생성
    feedbacks_filtered = [f for f in feedbacks if isinstance(f, str)]
    references_filtered = [r for r in references if isinstance(r, str)]

    embeddings1 = model.encode(feedbacks_filtered, convert_to_tensor=True)
    embeddings2 = model.encode(references_filtered, convert_to_tensor=True)

    # 코사인 유사도를 사용해 두 텍스트의 의미적 유사성 계산
    similarity_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

    # 결과 반환 (각 피드백-참조 쌍의 평균 유사도)
    return similarity_scores.mean().item()

In [ ]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)

In [ ]:
from tqdm import tqdm

def generate_feedback(df):
    """
    pandas DataFrame에서 job, question, answer 컬럼을 읽어와서,
    각각에 대한 피드백을 생성하고 진행률을 바 형태로 출력하는 함수.

    Parameters:
        df (pandas.DataFrame): job, question, answer 컬럼을 가진 데이터프레임

    Returns:
        pandas.DataFrame: 원래 데이터와 함께 피드백을 추가한 새로운 데이터프레임
    """

    # 결과 저장용 리스트
    feedback_list = []

    # 데이터프레임의 각 행에 대해 피드백 생성
    for index, row in tqdm(df.iterrows(), total=len(df), desc="진행률", ncols=100):
        job = row['직무']
        question = row['질문']
        answer = row['답변']

        # 메시지 구성
        messages = [
            {
                "role": "user",
                "content": (
                    f"자기소개서 문항에 대해서 지원자가 작성한 자기소개서 답변을 지원 직무를 고려하여, 채용 담당자 관점에서 개선점을 피드백 해주세요.\n"
                    f"지원 직무: {job}\n"
                    f"자기소개서 문항: {question}\n"
                    f"자기소개서 답변: {answer}"
                )
            }
        ]

        # 프롬프트 생성
        prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        # 피드백 생성
        outputs = pipe_finetuned(
            prompt,
            do_sample=True,
            temperature=0.2,
            top_k=50,
            top_p=0.95,
            add_special_tokens=True
        )

        # 프롬프트 부분을 제거하고 피드백 부분만 추출
        feedback = outputs[0]["generated_text"][len(prompt):]

        # 결과 리스트에 추가
        feedback_list.append(feedback)

    # 원래 데이터프레임에 피드백 컬럼 추가
    df['feedback'] = feedback_list

    return df


In [ ]:
generate_feedback(test_df)

In [ ]:
# 데이터프레임을 CSV 파일로 저장
test_df.to_csv(FEEDBACK_PATH, index=False, encoding="utf-8-sig")

In [ ]:
# feedbacks: df의 'feedback' 컬럼 (모델이 생성한 피드백)
feedbacks = test_df['feedback'].tolist()

# references: test_df의 '단점' 컬럼 (참조 텍스트)
references = test_df['단점'].tolist()

# STS 점수 계산
sts_score = evaluate_sts(feedbacks, references)
print("STS Score:", sts_score)

STS Score: 0.8379164934158325


# 5. 모델 업로드

In [ ]:
# 저장할 경로와 모델 리포지토리 이름
REPO_NAME = "your_repo_name"

## Upload to Huggingface Hub
finetune_model.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    use_auth_token=HUGGINGFACE_TOKEN
)
